In [ ]:
!pip3 install statsbombpy
!pip3 install mplsoccer

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.3/60.3 kB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.0/79.0 kB 1.1 MB/s eta 0:00:00


In [ ]:
from statsbombpy import sb
import matplotlib.pyplot as plt
from mplsoccer import Pitch, VerticalPitch
import pandas as pd
import xgboost
from xgboost import plot_importance
from xgboost import XGBClassifier

# Training Data Preperation

In [ ]:
# La-Liga Season 2020-2021 Analysis
#Training Dataset La-Liga
comp_id = 11
season_id = [90,42,4,1,2,27,26,25,24,23,22,21,41,40]
match_count = 0
train_dict = {}

for season in season_id:
    match_ids = sb.matches(competition_id=comp_id, season_id=season).match_id.unique()
    for match in match_ids:
        match_count = match_count + 1
        pass_events = sb.events(match_id=match, split=True, flatten_attrs=True)["passes"]

        # Group by possession and add a new column 'no_pass_possess' representing the rank within each group
        pass_events['no_pass_possess'] = pass_events.groupby('possession').cumcount() + 1

        match_data = {'events': pass_events}

        train_dict[str(match)] = match_data

train_df = pd.DataFrame()
for key in train_dict.keys():
  event = train_dict[key]['events']
  train_df = train_df.append(event)

In [ ]:
print(f"training Dataframe Specs \n Dataframe Shape: {train_df.shape} \n Dataset columns: {train_df.columns}")

In [ ]:
train_df['pass_outcome'].unique()

In [ ]:
train_df['pass_outcome'].fillna('Complete', inplace=True)
train_df.head(10)

In [ ]:
#Plot the distribution of unique values for each column
import seaborn as sns
import matplotlib.pyplot as plt
myCols = ['pass_type',
          'pass_height', 'pass_body_part', 'under_pressure',
          'pass_cross', 'play_pattern','pass_outcome',
          'pass_technique']

for col in myCols:
    plt.figure(figsize=(12, 6))
    sns.countplot(x=col, data=train_df, order=train_df[col].value_counts().index)
    plt.title(f'Distribution of {col}')
    plt.show()

### FEATURE SELECTION AND ENCODING

In [ ]:
myCols = ['location', 'pass_type','pass_length', 'pass_angle',
          'pass_height', 'pass_body_part', 'under_pressure',
          'pass_cross', 'play_pattern',
          'no_pass_possess', 'pass_outcome',
          'pass_technique']

train_df = train_df[myCols]
from scipy.sparse import data
# Preprocessing
from sklearn import preprocessing

data_model_train = train_df.copy()

data_model_train[['x', 'y']] = pd.DataFrame(train_df['location'].to_list(), index=train_df.index)

# Define a function to transform 'y' based on your condition
def transform_y(row):
    if row['y'] <= 40:
        return row['y']
    else:
        return 80 - row['y']

# Apply the function to create the new 'y_transformed' column
data_model_train['y'] = data_model_train.apply(transform_y, axis=1)
data_model_train = data_model_train.drop(['location'], axis=1)



#pass outcome
def label_encode(type):
  '''Function to encode pass outcome column.'''
  val = type['pass_outcome']
  #print(val)
  if str(val) == "Complete":
    return 1
  elif str(val) in ['Incomplete', 'Out', 'Pass Offside']:
    return 0
  else:
    return 2

#Body part
def label_encode2(type):
  '''Function to encode pass_body_part column.'''
  val = type['pass_body_part']
  #print(val)
  if str(val) == "Right Foot":
    return int(1)
  elif str(val) == "Left Foot":
    return int(2)
  elif str(val) == "Head":
    return int(3)
  elif str(val) == "Other":
    return int(4)
  else:
    return int(5)

#Under pressure
def label_encode3(type):
  '''Function to encode under_pressure column.'''
  val = type['under_pressure']
  #print(val)
  if str(val) == "True":
    return 1
  else:
    return 0

# pass cross
def label_encode4(type):
  '''Function to encode pass_cross column.'''
  val = type['pass_cross']
  #print(val)
  if str(val) == "True":
    return 1
  else:
    return 0


#pass technique
def label_encode5(type):
  '''Function to encode pass_technique column'''
  val = type['pass_technique']
  #print(val)
  if str(val) == "Straight":
    return 1
  elif str(val) == "Outswinging":
    return 2
  elif str(val) == "Inswinging":
    return 3
  elif str(val) == "Through Ball":
    return 4
  else:
    return 5


#play pattern
def label_encode6(type):
  '''Function to encode shot_technique column'''
  val = type['play_pattern']
  #print(val)
  if str(val) == "Regular Play":
    return 1
  elif str(val) == "From Kick Off":
    return 0
  elif str(val) == "From Throw In":
    return 0
  elif str(val) == "From Corner":
    return 0
  elif str(val) == "From Free Kick":
    return 0
  elif str(val) == "From Goal Kick":
    return 0
  elif str(val) == "From Keeper":
    return 0
  elif str(val) == "From Counter":
    return 1
  elif str(val) == "Other":
    return 2


label_encoder = preprocessing.LabelEncoder()
data_model_train['pass_type'] = label_encoder.fit_transform(data_model_train['pass_type'])
data_model_train['pass_height'] = label_encoder.fit_transform(data_model_train['pass_height'])

data_model_train['pass_outcome'] = data_model_train.apply(lambda row: label_encode(row), axis=1)
data_model_train['pass_body_part'] = data_model_train.apply(lambda row: label_encode2(row), axis=1)
data_model_train['under_pressure'] = data_model_train.apply(lambda row: label_encode3(row), axis=1)
data_model_train['pass_cross'] = data_model_train.apply(lambda row: label_encode4(row), axis=1)
data_model_train['pass_technique'] = data_model_train.apply(lambda row: label_encode5(row), axis=1)
data_model_train['play_pattern'] = data_model_train.apply(lambda row: label_encode6(row), axis=1)

In [ ]:
data_model_train.head(10)

,pass_type,pass_length,pass_angle,pass_height,pass_body_part,under_pressure,pass_cross,play_pattern,no_pass_possess,pass_outcome,pass_technique,x,y
0,4,12.210241,2.692596,0,2,0,0,0,1,1,5,61.0,39.9
1,7,15.471910,-1.720006,0,1,0,0,0,2,1,5,49.4,35.1
2,7,24.489998,2.111216,0,1,0,0,0,3,1,5,48.1,29.6
3,7,20.763910,-1.730403,0,1,0,0,0,4,1,5,44.3,29.8
4,7,10.023972,0.499347,0,2,0,0,0,5,1,5,49.0,28.7
5,7,23.129420,2.242837,0,1,0,0,0,6,1,5,57.1,33.1
6,7,24.148085,-1.851940,0,1,0,0,0,7,1,5,49.1,27.5
7,7,16.707483,-1.540865,0,2,1,0,0,8,1,5,41.3,29.2
8,7,14.483784,1.240312,0,1,1,0,0,9,1,5,37.1,14.6
9,7,33.092747,1.790121,0,2,0,0,0,10,1,5,42.4,27.6


# Testing Data Preperation

In [ ]:
#Testing Dataset La-Liga
comp_id = 16
season_id = [1,4,2,27,26,25,24,23,22,21,41,39,37,44]
match_count = 0
test_dict = {}

for season in season_id:
    match_ids = sb.matches(competition_id=comp_id, season_id=season).match_id.unique()
    for match in match_ids:
        match_count = match_count + 1
        pass_events = sb.events(match_id=match, split=True, flatten_attrs=True)["passes"]

        # Group by possession and add a new column 'no_pass_possess' representing the rank within each group
        pass_events['no_pass_possess'] = pass_events.groupby('possession').cumcount() + 1

        match_data = {'events': pass_events}

        test_dict[str(match)] = match_data

test_df = pd.DataFrame()
for key in test_dict.keys():
  event = test_dict[key]['events']
  test_df = test_df.append(event)

In [ ]:
print(f"training Dataframe Specs \n Dataframe Shape: {test_df.shape} \n Dataset columns: {test_df.columns}")

training Dataframe Specs 
 Dataframe Shape: (34588, 48) 
 Dataset columns: Index(['id', 'index', 'period', 'timestamp', 'minute', 'second', 'type',
       'possession', 'possession_team', 'play_pattern', 'team', 'player',
       'position', 'location', 'duration', 'related_events', 'match_id',
       'pass_recipient', 'pass_length', 'pass_angle', 'pass_height',
       'pass_end_location', 'pass_body_part', 'pass_type',
       'possession_team_id', 'team_id', 'player_id', 'pass_switch',
       'pass_outcome', 'under_pressure', 'pass_assisted_shot_id',
       'pass_shot_assist', 'pass_technique', 'pass_inswinging',
       'pass_through_ball', 'pass_outswinging', 'pass_aerial_won',
       'pass_cross', 'off_camera', 'pass_goal_assist', 'pass_no_touch',
       'pass_cut_back', 'counterpress', 'pass_straight', 'pass_deflected',
       'no_pass_possess', 'out', 'pass_miscommunication'],
      dtype='object')


In [ ]:
test_df['pass_outcome'].fillna('Complete', inplace=True)

In [ ]:
myCols = ['location', 'pass_type','pass_length', 'pass_angle',
          'pass_height', 'pass_body_part', 'under_pressure',
          'pass_cross', 'play_pattern',
          'no_pass_possess', 'pass_outcome',
          'pass_technique']

test_df = test_df[myCols]
from scipy.sparse import data
# Preprocessing
from sklearn import preprocessing

data_model_test = test_df.copy()

data_model_test[['x', 'y']] = pd.DataFrame(test_df['location'].to_list(), index=test_df.index)

# Define a function to transform 'y' based on your condition
def transform_y(row):
    if row['y'] <= 40:
        return row['y']
    else:
        return 80 - row['y']

# Apply the function to create the new 'y_transformed' column
data_model_test['y'] = data_model_test.apply(transform_y, axis=1)
data_model_test = data_model_test.drop(['location'], axis=1)

#pass outcome
def label_encode(type):
  '''Function to encode pass outcome column.'''
  val = type['pass_outcome']
  #print(val)
  if str(val) == "Complete":
    return 1
  elif str(val) in ['Incomplete', 'Out', 'Pass Offside']:
    return 0
  else:
    return 2

#Body part
def label_encode2(type):
  '''Function to encode pass_body_part column.'''
  val = type['pass_body_part']
  #print(val)
  if str(val) == "Right Foot":
    return int(1)
  elif str(val) == "Left Foot":
    return int(2)
  elif str(val) == "Head":
    return int(3)
  elif str(val) == "Other":
    return int(4)
  else:
    return int(5)

#Under pressure
def label_encode3(type):
  '''Function to encode under_pressure column.'''
  val = type['under_pressure']
  #print(val)
  if str(val) == "True":
    return 1
  else:
    return 0

# pass cross
def label_encode4(type):
  '''Function to encode pass_cross column.'''
  val = type['pass_cross']
  #print(val)
  if str(val) == "True":
    return 1
  else:
    return 0


#pass technique
def label_encode5(type):
  '''Function to encode pass_technique column'''
  val = type['pass_technique']
  #print(val)
  if str(val) == "Straight":
    return 1
  elif str(val) == "Outswinging":
    return 2
  elif str(val) == "Inswinging":
    return 3
  elif str(val) == "Through Ball":
    return 4
  else:
    return 5


#play pattern
def label_encode6(type):
  '''Function to encode shot_technique column'''
  val = type['play_pattern']
  #print(val)
  if str(val) == "Regular Play":
    return 1
  elif str(val) == "From Kick Off":
    return 0
  elif str(val) == "From Throw In":
    return 0
  elif str(val) == "From Corner":
    return 0
  elif str(val) == "From Free Kick":
    return 0
  elif str(val) == "From Goal Kick":
    return 0
  elif str(val) == "From Keeper":
    return 0
  elif str(val) == "From Counter":
    return 1
  elif str(val) == "Other":
    return 2


label_encoder = preprocessing.LabelEncoder()
data_model_test['pass_type'] = label_encoder.fit_transform(data_model_test['pass_type'])
data_model_test['pass_height'] = label_encoder.fit_transform(data_model_test['pass_height'])

data_model_test['pass_outcome'] = data_model_test.apply(lambda row: label_encode(row), axis=1)
data_model_test['pass_body_part'] = data_model_test.apply(lambda row: label_encode2(row), axis=1)
data_model_test['under_pressure'] = data_model_test.apply(lambda row: label_encode3(row), axis=1)
data_model_test['pass_cross'] = data_model_test.apply(lambda row: label_encode4(row), axis=1)
data_model_test['pass_technique'] = data_model_test.apply(lambda row: label_encode5(row), axis=1)
data_model_test['play_pattern'] = data_model_test.apply(lambda row: label_encode6(row), axis=1)

In [ ]:
data_model_train.head(10)

,pass_type,pass_length,pass_angle,pass_height,pass_body_part,under_pressure,pass_cross,play_pattern,no_pass_possess,pass_outcome,pass_technique,x,y
0,4,12.210241,2.692596,0,2,0,0,0,1,1,5,61.0,39.9
1,7,15.471910,-1.720006,0,1,0,0,0,2,1,5,49.4,35.1
2,7,24.489998,2.111216,0,1,0,0,0,3,1,5,48.1,29.6
3,7,20.763910,-1.730403,0,1,0,0,0,4,1,5,44.3,29.8
4,7,10.023972,0.499347,0,2,0,0,0,5,1,5,49.0,28.7
5,7,23.129420,2.242837,0,1,0,0,0,6,1,5,57.1,33.1
6,7,24.148085,-1.851940,0,1,0,0,0,7,1,5,49.1,27.5
7,7,16.707483,-1.540865,0,2,1,0,0,8,1,5,41.3,29.2
8,7,14.483784,1.240312,0,1,1,0,0,9,1,5,37.1,14.6
9,7,33.092747,1.790121,0,2,0,0,0,10,1,5,42.4,27.6


In [ ]:
nan_counts = data_model_train.isna().sum()

# Display the NaN counts for each column
print(nan_counts)

pass_type          0
pass_length        0
pass_angle         0
pass_height        0
pass_body_part     0
under_pressure     0
pass_cross         0
play_pattern       0
no_pass_possess    0
pass_outcome       0
pass_technique     0
x                  0
y                  0
dtype: int64


In [ ]:
for column in data_model_train.columns:
    unique_values = data_model_train[column].unique()
    print(f'Column: {column}, Unique Values: {unique_values}')

# Logistic Regression

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [ ]:
training = data_model_train.sample(frac=1).reset_index(drop=True)
features = training[['x', 'y', 'pass_type','pass_length', 'pass_angle',
                    'pass_height', 'pass_body_part', 'under_pressure',
                    'pass_cross', 'play_pattern', 'no_pass_possess',
                    'pass_technique']]

target = training[['pass_outcome']]

scaled = StandardScaler()
X_scaled = scaled.fit_transform(features)

all_data = X_scaled

X_train = all_data
y_train = target

print('Training features shape', X_train.shape)
print('Training target shape', y_train.shape)

Training features shape (153745, 12)
Training target shape (153745, 1)


In [ ]:
testing = data_model_test.sample(frac=1).reset_index(drop=True)
features_test = testing[['x', 'y', 'pass_type','pass_length', 'pass_angle',
                    'pass_height', 'pass_body_part', 'under_pressure',
                    'pass_cross', 'play_pattern', 'no_pass_possess',
                    'pass_technique']]

target_test = testing[['pass_outcome']]

scaled = StandardScaler()
X_scaled_test = scaled.fit_transform(features_test)

all_data_test = X_scaled_test

X_test = all_data_test
y_test = target_test

print('Training features shape', X_test.shape)
print('Training target shape', y_test.shape)

Training features shape (34588, 12)
Training target shape (34588, 1)


In [ ]:
model = LogisticRegression(fit_intercept=True, max_iter=20000)
model.fit(X_train,  y_train)

#Model testing
predicted_classes1_0 = model.predict(X_test)
accuracy1 = accuracy_score(y_test,predicted_classes1_0)
p_pred1 = model.predict_proba(X_test)

print("************{GOAL SCORED VS [ANGLE, DISTANCE]**************")
print('Model accuracy on la-liga data: ', accuracy1)
print('intercept:', model.intercept_)
print('coef:', model.coef_, end='\n\n')
p_pred1[:,1].shape


************{GOAL SCORED VS [ANGLE, DISTANCE]**************
Model accuracy on la-liga data:  0.8303457846652018
intercept: [ 0.48427521  2.47216149 -2.9564367 ]
coef: [[ 0.13214125  0.04618512  0.01610796  0.11124038 -0.00216717  0.21133828
  -0.01104454  0.08383967  0.08350085 -0.0259304   0.10364498 -0.05780527]
 [ 0.00207632  0.03137637  0.05375872 -0.23185237 -0.00042399 -0.38111081
   0.00930592 -0.13163029 -0.17443202  0.04057364  0.01276368  0.09826687]
 [-0.13421756 -0.07756149 -0.06986668  0.12061199  0.00259116  0.16977253
   0.00173862  0.04779062  0.09093118 -0.01464324 -0.11640866 -0.04046161]]



(34588,)

# Other Work

In [ ]:
# La-Liga Season 2020-2021 Analysis
#Training Dataset La-Liga
comp_id = 11
season_id = [90,42,4]
match_count = 0
test_dict = {}

for season in season_id:
  #print(season)
  match_ids = sb.matches(competition_id=comp_id, season_id=season).match_id.unique()
  for match in match_ids:
    match_count = match_count+1
    pass_events = sb.events(match_id=match, split=True, flatten_attrs=True)["passes"]
    match_data = {'events': pass_events}
    test_dict[str(match)] = match_data

df = pd.DataFrame()
for key in test_dict.keys():
  event = test_dict[key]['events']
  df = df.append(event)

print(f"training Dataframe Specs \n Dataframe Shape: {df.shape} \n Dataset columns: {df.columns}")

In [ ]:
df['pass_outcome'].fillna('Complete', inplace=True)
print('*******La-Liga Analysis******')
print('Total Season:', len(season_id))
print('Total Matches: ', match_count)
print('Total Pass: ', df.shape[0])
made = df[df['pass_outcome'] == 'Complete']
miss = df[(df['pass_outcome'] == 'Incomplete') | (df['pass_outcome'] == 'Out') | (df['pass_outcome'] == 'Pass Offside')]
print('Total completed pass : ', made.shape[0])
print('Total uncompleted pass : ', miss.shape[0])

*******La-Liga Analysis******
Total Season: 3
Total Matches:  102
Total Pass:  114849
Total completed pass :  97644
Total uncompleted pass :  16631


In [ ]:
made.head(5)

,id,index,period,timestamp,minute,second,type,possession,possession_team,play_pattern,...,pass_assisted_shot_id,pass_shot_assist,pass_no_touch,pass_cut_back,pass_inswinging,counterpress,pass_straight,pass_goal_assist,pass_miscommunication,out
0,208f072d-cffa-4575-bd73-e9e9384c8fe1,5,1,00:00:00.689,0,0,Pass,2,Barcelona,From Kick Off,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,e1f11e14-fb24-4c82-868f-507385122b9c,7,1,00:00:01.653,0,1,Pass,2,Barcelona,From Kick Off,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,4d801bc4-907a-4474-9620-7c6ca2610ea7,10,1,00:00:05.344,0,5,Pass,2,Barcelona,From Kick Off,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,72a0c004-6d76-4ea8-b892-0f7881be9320,13,1,00:00:10.283,0,10,Pass,2,Barcelona,From Kick Off,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,ba9d104d-bddd-44bd-b156-991e7f5f0344,16,1,00:00:14.201,0,14,Pass,2,Barcelona,From Kick Off,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Columns we are interested in


1.  X-coordinate of pass start location
2.  Mirrored y-coordinate of pass start location
3.  angle of pass
4. Distance
5. Was it played with feet or head
6. Was it a cross
7. Was it in open play or set piece
8. Was it a ground pass or airborne

More advance features
9. How many passes were in the possession so far
10. How far was the ball carried by the player beforehand
11. How fast has the ball moved vertically up the pitch in the previous three action
12. was the previous action by a teammate or an opponent.
13. what kind of event was the previous action
14. what was the distance and angle of the previous pass





In [ ]:
myCols = ['location', 'pass_type','pass_length', 'pass_angle', 'pass_height', 'pass_body_part', 'under_pressure', 'pass_cross', 'play_pattern', 'possession', 'pass_outcome', 'pass_technique']

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 114849 entries, 0 to 1060
Data columns (total 47 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   id                     114849 non-null  object 
 1   index                  114849 non-null  int64  
 2   period                 114849 non-null  int64  
 3   timestamp              114849 non-null  object 
 4   minute                 114849 non-null  int64  
 5   second                 114849 non-null  int64  
 6   type                   114849 non-null  object 
 7   possession             114849 non-null  int64  
 8   possession_team        114849 non-null  object 
 9   play_pattern           114849 non-null  object 
 10  team                   114849 non-null  object 
 11  player                 114849 non-null  object 
 12  position               114849 non-null  object 
 13  location               114849 non-null  object 
 14  duration               114849 non-null

Columns I am interested in
Location
End Location
Play Pattern
Pass Length
Pass Angle
Pass Height
Pass Body part
Pass Type
P


In [ ]:
df = df[myCols]
df.head()

,location,pass_type,pass_length,pass_angle,pass_height,pass_body_part,under_pressure,pass_cross,play_pattern,possession,pass_outcome,pass_technique
0,"[61.0, 40.1]",Kick Off,12.210241,2.692596,Ground Pass,Left Foot,NaN,NaN,From Kick Off,2,Complete,NaN
1,"[49.4, 44.9]",NaN,15.471910,-1.720006,Ground Pass,Right Foot,NaN,NaN,From Kick Off,2,Complete,NaN
2,"[48.1, 29.6]",NaN,24.489998,2.111216,Ground Pass,Right Foot,NaN,NaN,From Kick Off,2,Complete,NaN
3,"[44.3, 50.2]",NaN,20.763910,-1.730403,Ground Pass,Right Foot,NaN,NaN,From Kick Off,2,Complete,NaN
4,"[49.0, 28.7]",NaN,10.023972,0.499347,Ground Pass,Left Foot,NaN,NaN,From Kick Off,2,Complete,NaN


In [ ]:
from scipy.sparse import data
# Preprocessing
from sklearn import preprocessing

data_model = df.copy()


data_model[['x', 'y']] = pd.DataFrame(df['location'].to_list(), index=df.index)

# Define a function to transform 'y' based on your condition
def transform_y(row):
    if row['y'] <= 40:
        return row['y']
    else:
        return 80 - row['y']

# Apply the function to create the new 'y_transformed' column
data_model['y'] = data_model.apply(transform_y, axis=1)
data_model = data_model.drop(['location'], axis=1)

#pass outcome
def label_encode(type):
  '''Function to encode pass outcome column.'''
  val = type['pass_outcome']
  #print(val)
  if str(val) == "Complete":
    return 1
  else:
    return 0

#Body part
def label_encode2(type):
  '''Function to encode pass_body_part column.'''
  val = type['pass_body_part']
  #print(val)
  if str(val) == "Right Foot":
    return 1
  elif str(val) == "Left Foot":
    return 2
  elif str(val) == "Head":
    return 3
  elif str(val) == "Other":
    return 4

#Under pressure
def label_encode3(type):
  '''Function to encode under_pressure column.'''
  val = type['under_pressure']
  #print(val)
  if str(val) == "True":
    return 1
  else:
    return 0

# pass cross
def label_encode4(type):
  '''Function to encode pass_cross column.'''
  val = type['pass_cross']
  #print(val)
  if str(val) == "True":
    return 1
  else:
    return 0


#pass technique
def label_encode5(type):
  '''Function to encode pass_technique column'''
  val = type['pass_technique']
  #print(val)
  if str(val) == "Straight":
    return 1
  elif str(val) == "Outswinging":
    return 2
  elif str(val) == "Inswinging":
    return 3
  elif str(val) == "Through Ball":
    return 4
  else:
    return 5


#play pattern
def label_encode6(type):
  '''Function to encode shot_technique column'''
  val = type['play_pattern']
  #print(val)
  if str(val) == "Regular Play":
    return 1
  elif str(val) == "From Kick Off":
    return 2
  elif str(val) == "From Throw In":
    return 3
  elif str(val) == "From Corner":
    return 4
  elif str(val) == "From Free Kick":
    return 5
  elif str(val) == "From Goal Kick":
    return 6
  elif str(val) == "From Keeper":
    return 7
  elif str(val) == "From Counter":
    return 8
  elif str(val) == "Other":
    return 9


label_encoder = preprocessing.LabelEncoder()
data_model['pass_type'] = label_encoder.fit_transform(data_model['pass_type'])
data_model['pass_height'] = label_encoder.fit_transform(data_model['pass_height'])

data_model['pass_outcome'] = data_model.apply(lambda row: label_encode(row), axis=1)
data_model['pass_body_part'] = data_model.apply(lambda row: label_encode2(row), axis=1)
data_model['under_pressure'] = data_model.apply(lambda row: label_encode3(row), axis=1)
data_model['pass_cross'] = data_model.apply(lambda row: label_encode4(row), axis=1)
data_model['pass_technique'] = data_model.apply(lambda row: label_encode5(row), axis=1)
data_model['play_pattern'] = data_model.apply(lambda row: label_encode6(row), axis=1)


In [ ]:
data_model.head()

,pass_type,pass_length,pass_angle,pass_height,pass_body_part,under_pressure,pass_cross,play_pattern,possession,pass_outcome,pass_technique,x,y
0,4,12.210241,2.692596,0,2.0,0,0,2,2,1,5,61.0,39.9
1,7,15.471910,-1.720006,0,1.0,0,0,2,2,1,5,49.4,35.1
2,7,24.489998,2.111216,0,1.0,0,0,2,2,1,5,48.1,29.6
3,7,20.763910,-1.730403,0,1.0,0,0,2,2,1,5,44.3,29.8
4,7,10.023972,0.499347,0,2.0,0,0,2,2,1,5,49.0,28.7


In [ ]:
data_model.pass_type.unique()

array([4, 7, 5, 6, 0, 1, 2, 3])

In [ ]:
data_model.pass_technique.unique()

array([5, 4, 2, 3, 1])

In [ ]:
data_model.pass_outcome.unique()

array([1, 0])

In [ ]:
data_model.pass_body_part.unique()

array([ 2.,  1., nan,  3.,  4.])

In [ ]:
data_model.under_pressure.unique()

array([0, 1])

In [ ]:
data_model.play_pattern.unique()

array([2, 1, 3, 4, 5, 6, 7, 9, 8])

In [ ]:
data_model.pass_cross.unique()

array([0, 1])

In [ ]:
data_model.pass_height.unique()

array([0, 2, 1])

In [ ]:
df.possession.unique()

array([  2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,  13,  14,
        15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,  26,  27,
        28,  29,  30,  31,  32,  34,  35,  36,  38,  39,  40,  41,  42,
        43,  44,  45,  46,  47,  48,  49,  50,  51,  52,  53,  54,  55,
        56,  57,  58,  59,  60,  62,  65,  66,  67,  68,  69,  70,  71,
        72,  73,  75,  76,  77,  78,  79,  80,  81,  82,  84,  85,  86,
        87,  88,  89,  90,  91,  92,  93,  94,  95,  96,  97,  99, 100,
       101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113,
       114, 115, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127,
       128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140,
       141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153,
       154, 155, 156, 157, 158, 159, 160, 161, 162, 163,  33,  61,  63,
        64,  83,  98, 116,  37,  74, 164, 165, 166, 167, 168, 169, 170,
       172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 18

In [ ]:
df.head(50)

,location,pass_type,pass_length,pass_angle,pass_height,pass_body_part,under_pressure,pass_cross,play_pattern,possession,pass_outcome
0,"[61.0, 40.1]",Kick Off,12.210241,2.692596,Ground Pass,Left Foot,NaN,NaN,From Kick Off,2,Complete
1,"[49.4, 44.9]",NaN,15.471910,-1.720006,Ground Pass,Right Foot,NaN,NaN,From Kick Off,2,Complete
2,"[48.1, 29.6]",NaN,24.489998,2.111216,Ground Pass,Right Foot,NaN,NaN,From Kick Off,2,Complete
3,"[44.3, 50.2]",NaN,20.763910,-1.730403,Ground Pass,Right Foot,NaN,NaN,From Kick Off,2,Complete
4,"[49.0, 28.7]",NaN,10.023972,0.499347,Ground Pass,Left Foot,NaN,NaN,From Kick Off,2,Complete
5,"[57.1, 33.1]",NaN,23.129420,2.242837,Ground Pass,Right Foot,NaN,NaN,From Kick Off,2,Complete
6,"[49.1, 52.5]",NaN,24.148085,-1.851940,Ground Pass,Right Foot,NaN,NaN,From Kick Off,2,Complete
7,"[41.3, 29.2]",NaN,16.707483,-1.540865,Ground Pass,Left Foot,True,NaN,From Kick Off,2,Complete
8,"[37.1, 14.6]",NaN,14.483784,1.240312,Ground Pass,Right Foot,True,NaN,From Kick Off,2,Complete
9,"[42.4, 27.6]",NaN,33.092747,1.790121,Ground Pass,Left Foot,NaN,NaN,From Kick Off,2,Complete


In [ ]:
poss = df.groupby('possession')
for group_key, group_df in poss:
    print(f'Group {group_key}: Shape {group_df.shape}')

Group 2: Shape (1318, 11)
Group 3: Shape (1018, 11)
Group 4: Shape (630, 11)
Group 5: Shape (913, 11)
Group 6: Shape (723, 11)
Group 7: Shape (626, 11)
Group 8: Shape (641, 11)
Group 9: Shape (753, 11)
Group 10: Shape (764, 11)
Group 11: Shape (741, 11)
Group 12: Shape (739, 11)
Group 13: Shape (565, 11)
Group 14: Shape (778, 11)
Group 15: Shape (759, 11)
Group 16: Shape (709, 11)
Group 17: Shape (691, 11)
Group 18: Shape (665, 11)
Group 19: Shape (810, 11)
Group 20: Shape (754, 11)
Group 21: Shape (667, 11)
Group 22: Shape (702, 11)
Group 23: Shape (736, 11)
Group 24: Shape (737, 11)
Group 25: Shape (655, 11)
Group 26: Shape (725, 11)
Group 27: Shape (654, 11)
Group 28: Shape (591, 11)
Group 29: Shape (653, 11)
Group 30: Shape (559, 11)
Group 31: Shape (631, 11)
Group 32: Shape (668, 11)
Group 33: Shape (786, 11)
Group 34: Shape (661, 11)
Group 35: Shape (799, 11)
Group 36: Shape (655, 11)
Group 37: Shape (728, 11)
Group 38: Shape (661, 11)
Group 39: Shape (618, 11)
Group 40: Shape (7

In [ ]:
poss2 = poss.get_group(2)
poss2.head(100)

,location,pass_type,pass_length,pass_angle,pass_height,pass_body_part,under_pressure,pass_cross,play_pattern,possession,pass_outcome
0,"[61.0, 40.1]",Kick Off,12.210241,2.692596,Ground Pass,Left Foot,NaN,NaN,From Kick Off,2,Complete
1,"[49.4, 44.9]",NaN,15.471910,-1.720006,Ground Pass,Right Foot,NaN,NaN,From Kick Off,2,Complete
2,"[48.1, 29.6]",NaN,24.489998,2.111216,Ground Pass,Right Foot,NaN,NaN,From Kick Off,2,Complete
3,"[44.3, 50.2]",NaN,20.763910,-1.730403,Ground Pass,Right Foot,NaN,NaN,From Kick Off,2,Complete
4,"[49.0, 28.7]",NaN,10.023972,0.499347,Ground Pass,Left Foot,NaN,NaN,From Kick Off,2,Complete
...,...,...,...,...,...,...,...,...,...,...,...
2,"[49.9, 52.6]",NaN,20.155893,1.645285,Ground Pass,Left Foot,NaN,NaN,From Kick Off,2,Complete
3,"[47.5, 70.3]",NaN,15.239751,-2.286540,Ground Pass,Right Foot,NaN,NaN,From Kick Off,2,Complete
4,"[37.9, 53.9]",NaN,33.653680,-1.627284,Ground Pass,Right Foot,NaN,NaN,From Kick Off,2,Complete
5,"[35.3, 20.3]",NaN,28.058510,1.706645,Ground Pass,Left Foot,NaN,NaN,From Kick Off,2,Complete
